In [1]:
import re
import time as t
import pandas as pd
import pickle as pk
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from bs4 import BeautifulSoup

In [2]:
STRAVA_URL = 'https://www.strava.com'
password = r'1357WorkWork!'
p = '1357Worker!'
usernames = [
    r'kvspek@gmail.com',
    r'spektork@post.bgu.ac.il',
    r'kev_backup@yahoo.com',
    r'cool_table@yahoo.com',
    r'cool_table2@yahoo.com',
    r'cool_table3@yahoo.com',
    r'cool_table4@yahoo.com',
    r'cool_table5@yahoo.com',
]

username_backup = r'kev_backup@yahoo.com'
password = r'1357Backup!'

## Prepare chrome driver and sign in to Strava

In [3]:
def chrome_driver(username, password):
    options = webdriver.ChromeOptions()
    options.add_argument('headless')

    # start chrome browser
    browser = webdriver.Chrome(options=options)

    browser.get(STRAVA_URL + '/login')
    email = browser.find_element_by_id("email")
    password = browser.find_element_by_id("password")

    email.send_keys(username)
    password.send_keys(password)
    
    browser.find_element_by_id("login-button").click()
    print(browser.current_url)
    return browser

## Extract rider activities

In [4]:
 class Rider:
    def __init__(self, rider_name, rider_url, rider_id):
        self.rider_name = rider_name.lower()
        self.rider_url = rider_url
        self.rider_id = rider_id
        self.activity_links = set()
        self.links = []
        
        # tables
        self.workout = []
        self.workout_hrs = []
        self.workout_cadences = []
        self.workout_powers = []
        self.workout_speeds = []
        
    def __repr__(self):
        return f'Rider_link: {self.rider_url}, Rider name: {self.rider_name}, Rider ID: {self.rider_id}'

In [33]:
class GetActivities(): 
    
    def __init__(self, username, riders, years = list(range(2015,2022)), months = list(range(1,13))):
        self.activity_links = set()
        self.riders = riders
        self.years = years
        self.months = months
        self.username = username
        self.STRAVA_URL = 'https://www.strava.com'
        
    def _open_driver(self):
        
        options = webdriver.ChromeOptions()
        options.add_argument('headless')
        
        self.browser = webdriver.Chrome(options=options)
        URL = self.STRAVA_URL + '/login'
        
        self.browser.get(URL)
        email = self.browser.find_element_by_id("email")
        password = self.browser.find_element_by_id("password")

        email.send_keys(self.username)
        password.send_keys('12345678')
        
        self.browser.find_element_by_id("login-button").click()
        t.sleep(1)
        print(self.browser.current_url)
        
    def _close_driver(self):
        self.browser.close()
        
    def extract_activities(self):
        
        self._open_driver()
        problematic_riders = []
        for rider in self.riders:
            
            self.browser.get(rider.rider_url)
            t.sleep(1)
            
            print(rider)
            try:
                # Click on 'monthly' button so that the date inteverals would be smaller
                monthly_xpath = '//*[@id="interval-graph-controls"]/ul/li[2]/ul/li[2]/a'
                interval_xpath = '//*[@id="interval-graph-columns"]/div/ul[2]'

                soup = BeautifulSoup(self.browser.page_source, 'html.parser')
                date_intervals = soup.find('div', {'class': 'drop-down-menu drop-down-sm enabled'})
                temp_links = date_intervals.find_all('a')

                date_links = []
                for temp_link in temp_links:
                    link = STRAVA_URL + temp_link['href']
                    link = link.replace('week', 'month')
                    date_links.append(link)


                current_link_year = soup.find('h2', {'class': 'text-callout left'}).text[-5:-1]
                current_link = date_links[0]
                interval_exmaple = re.search("interval=.*&", current_link).group(0)
                current_link = current_link.replace(interval_exmaple[-7:-3], current_link_year)
                current_link = current_link[:-1] + '0'
                date_links.insert(0, current_link)

                self.links = []
                interval_soups = []
                prev_url = self.browser.current_url
                for link in date_links:

                    interval1 = re.search("interval=.*&", link).group(0)
                    year = int(interval1[-7:-3])

                    if year-1 in self.years:

                        for month in range(7,13):
                            if month not in self.months:
                                continue

                            month_string = str(month) if month >= 10 else f'0{month}'
                            interval2 = interval1.replace(str(year)+interval1[-3:-1], str(year-1)+month_string)
                            rider.links.append(link.replace(interval1, interval2))

                    if year in self.years:

                        for month in range(1,8):
                            if month not in self.months:
                                continue

                            interval2 = interval1.replace(str(year)+interval1[-3:-1], str(year)+f'0{month}')
                            rider.links.append(link.replace(interval1, interval2))
                            
            except:
                problematic_riders.append(rider)
            
        self.riders = [rider for rider in self.riders if rider not in problematic_riders]  
        self._close_driver()

    def run(self):
        
        self._open_driver()

       
        for rider in self.riders:
            prev = set()
            curr = set()
            for link in rider.links:
                
                
                year_link = int(re.search('interval=.*&', link).group(0)[9:-3])
                if year_link not in self.years:
                    continue
                
               
                self.browser.get(link)
                site_soup = BeautifulSoup(self.browser.page_source, 'html.parser')

                if site_soup.find('html')['class'][0] == 'logged-out': # logged out ...
                    print(f'logged out...')
                    self._close_driver()
                    return

                #print(f'{self.browser.current_url}') 
                t.sleep(2)
                timeout = t.time() + 10
                new_curr = set()

                while curr == prev:
                    t.sleep(1)
                    graph_soup = BeautifulSoup(self.browser.page_source, 'html.parser')
                    group_feed_activities = graph_soup.find_all('div', {'class': 'feed-entry group-activity'})
                    normal_activities = graph_soup.find_all('div', {'class': 'activity feed-entry entity-details'})

                    for group_feed_activity in group_feed_activities:
                        # loop through all group activities
                        group_feed_entry_body = group_feed_activity.find('div', {'class': 'entry-body'})
                        group_feed_link = group_feed_entry_body.find('a')
                        try:
                            new_curr.add(self.STRAVA_URL + group_feed_link.get('href'))
                        except:
                            continue
                        
                    for normal_activity in normal_activities:
                        # loop through all normal activities
                        normal_activity_entry_body = normal_activity.find('div', {'class': 'entry-body'})
                        normal_activity_link = normal_activity_entry_body.find('a')
                        try:
                            new_curr.add(self.STRAVA_URL + normal_activity_link.get('href'))
                        except:
                            continue
                    curr = new_curr

                    if t.time() > timeout:
                        break

                prev = curr

                rider.activity_links = rider.activity_links.union(curr)
            print(f'finished rider: {rider.rider_id}, number of activities: {len(rider.activity_links)}')
            
        self._close_driver()


In [6]:
def valid_rider_url(url):
    pattern = re.compile("https://www.strava.com/[a-zA-Z]+/[0-9]+$")
    found = pattern.search(url)
    if not found:
        return False
    return True

## Extract Activity Links 

In [10]:
activity_links = set()

In [7]:
df = pd.read_csv('data/Fetch_url_cyclists.csv')
df = df[df['url'].notna()]
riders = []
for index, row in df.iterrows():
    rider = Rider(row['full_name'], row['url'], row['cyclist_id'])
    if valid_rider_url(rider.rider_url):
        riders.append(rider)
        
print(len(riders))

riders_pickle =  open('data/riders.pickle', 'rb')
riders_extracted = pk.load(riders_pickle)
riders_extracted_ids = [rider.rider_id for rider in riders_extracted]
print(len(riders_extracted))
riders = [rider for rider in riders if rider.rider_id not in riders_extracted_ids]
print(len(riders)) # -> problamatic riders...

593
547
46


### Extract links to fetch from the activities links

In [8]:
get_activities = GetActivities(usernames[1], riders)
get_activities.extract_activities()

https://www.strava.com/onboarding
Rider_link: https://www.strava.com/athletes/12337203, Rider name: arashiro yukiya, Rider ID: 26
Rider_link: https://www.strava.com/athletes/4419459, Rider name: badilatti matteo, Rider ID: 45
Rider_link: https://www.strava.com/athletes/26007448, Rider name: bartolozzi andrea, Rider ID: 64
Rider_link: https://www.strava.com/athletes/3409968, Rider name: ben moshe yuval, Rider ID: 78
Rider_link: https://www.strava.com/athletes/13308515, Rider name: berrade urko, Rider ID: 90
Rider_link: https://www.strava.com/athletes/41905231, Rider name: bohli tom, Rider ID: 112
Rider_link: https://www.strava.com/athletes/4119801, Rider name: chaves esteban, Rider ID: 181
Rider_link: https://www.strava.com/athletes/20388790, Rider name: cherkasov nikolay, Rider ID: 183
Rider_link: https://www.strava.com/athletes/84189953, Rider name: de buyst jasper, Rider ID: 229
Rider_link: https://www.strava.com/pros/1005431, Rider name: earle nathan, Rider ID: 283
Rider_link: https

### Save riders with pickle

In [9]:
riders_pickle =  open('data/riders.pickle', 'rb')
riders_missing = get_activities.riders
riders = pk.load(riders_pickle)
riders = riders + riders_missing
riders.sort(key=lambda x: x.rider_id)
print(len(riders))

553


In [10]:
with open('data/riders.pickle', 'wb') as handle:
    pk.dump(riders, handle, protocol=pk.HIGHEST_PROTOCOL)
    

### Extract all activities from the link previously fetched

In [11]:
# filter ISN riders
isn_riders_id = list(pd.read_csv("data/ISN_cyclists.csv").cyclist_id)
riders_pickle =  open('data/riders.pickle', 'rb')
riders_extracted_pickle = open('data/isn_riders_2021_to_2018.pickle', 'rb')
riders = pk.load(riders_pickle)
riders_extracted = pk.load(riders_extracted_pickle)

temp = []
for rider in riders:
    if rider.rider_id in isn_riders_id:
        temp.append(rider)
riders = temp

isn_ids_extracted = [rider.rider_id for rider in riders_extracted]
print(len(isn_ids_extracted))
riders_missing = [rider for rider in riders if rider.rider_id not in isn_ids_extracted]

len(riders_missing)

35


6

In [44]:
isn_riders_pickle = open('data/isn_riders_2021_to_2018.pickle', 'rb')
isn_riders = pk.load(isn_riders_pickle)
len(isn_riders)

41

In [47]:
get_activities = GetActivities(usernames[1], isn_riders, years=[2015,2016,2017])

In [48]:
get_activities.run()

https://www.strava.com/onboarding
finished rider: 78, number of activities: 1580
finished rider: 283, number of activities: 413
finished rider: 291, number of activities: 1304
finished rider: 590, number of activities: 327
finished rider: 624, number of activities: 119
finished rider: 678, number of activities: 677
finished rider: 40, number of activities: 1366
finished rider: 56, number of activities: 201
finished rider: 101, number of activities: 130
finished rider: 114, number of activities: 1891
finished rider: 130, number of activities: 839
finished rider: 160, number of activities: 1136
finished rider: 170, number of activities: 1425
finished rider: 266, number of activities: 1765
finished rider: 275, number of activities: 1728
finished rider: 326, number of activities: 212
finished rider: 367, number of activities: 321
finished rider: 377, number of activities: 1373
finished rider: 411, number of activities: 133
finished rider: 427, number of activities: 387
finished rider: 456,

In [51]:
riders_extracted_pickle = open('data/isn_riders.pickle', 'rb')
riders_extracted = pk.load(riders_extracted_pickle)
len(riders_extracted)

41

In [27]:
riders = get_activities.riders + riders_extracted
len(riders)

41

In [50]:
total_activities = 0
for rider in get_activities.riders:
    total_activities += len(rider.activity_links)
total_activities

42545

In [49]:
with open('data/isn_riders.pickle', 'wb') as handle:
    pk.dump(get_activities.riders, handle, protocol=pk.HIGHEST_PROTOCOL)

In [9]:
with open('data/riders_2021_to_2018.pickle', 'wb') as handle:
    pk.dump(get_activities.riders, handle, protocol=pk.HIGHEST_PROTOCOL)

In [17]:
r 

{'https://www.strava.com/activities/1713376858', 'https://www.strava.com/activities/1700189665', 'https://www.strava.com/activities/1727428361', 'https://www.strava.com/activities/1735660156', 'https://www.strava.com/activities/1704519839'}
https://www.strava.com/pros/2276607


## Finish

In [82]:
pickle = pickle.dumps(get_activities.riders)

In [88]:
with open('data/riders.pickle', 'wb') as handle:
    pk.dump(get_activities.riders, handle, protocol=pk.HIGHEST_PROTOCOL)

In [89]:
with open('data/riders.pickle', 'rb') as handle:
    riders = pk.load(handle)

433
